In [217]:
#from IPython.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))
#from IPython.display import clear_output
#import time
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym
from scipy import integrate
from tqdm import tqdm
sym.init_session(use_latex=True)

IPython console for SymPy 1.9 (Python 3.9.7-64-bit) (ground types: python)

These commands were executed:
>>> from __future__ import division
>>> from sympy import *
>>> x, y, z, t = symbols('x y z t')
>>> k, m, n = symbols('k m n', integer=True)
>>> f, g, h = symbols('f g h', cls=Function)
>>> init_printing()

Documentation can be found at https://docs.sympy.org/1.9/



# Parcial 3

# 22.

In [218]:
def mulM(A,B,type=np.float64):

    if len(np.shape(A)) == 1:
        A = np.reshape(A,(1,len(A)))

    if len(np.shape(B)) == 1:
        B = np.reshape(B,(len(B),1))

    AB = np.zeros(shape=(np.shape(A)[0],np.shape(B)[1]),dtype=type)

    if np.shape(A)[1] == np.shape(B)[0]:
        for i in range(np.shape(B)[1]):
            for j in range(np.shape(A)[0]):
                suma = 0
                for k in range(np.shape(A)[1]):
                    suma += A[j][k]*B[k][i]  
                AB[j][i] = suma
    else:
        AB = None
        print('Multiplicación no está definida')
    
    return AB

def NewtonRaphson(z0,Fn,Jn,itmax=1000,error=1e-7):
    
    it = 0
    d = 1.
    
    while d > error and it < itmax:
        
        # Vector actual
        rc = z0

        InvJ = np.linalg.inv(Jn(rc[0],rc[1]))
        
        z0 = rc - np.reshape(mulM(InvJ,Fn(rc[0],rc[1])),-1)
        
        d = np.max(np.abs(z0 - rc))
        
        it += 1
    
    return np.round(z0,7)

def GetRoots(z0,Fn,Jn,tolerancia = 7):
    
    Roots = []
    
    for i in z0:
        
        root = NewtonRaphson(i,Fn,Jn)
        #print(root)

        if  type(root)!=bool:
            croot = tuple(np.round(root,tolerancia))

            if croot not in Roots:
                Roots.append(croot)
    Roots.sort()
    
    return Roots

In [219]:
w = 114.6
q = 3./10000.
k = 9000000000.
l = 5.

In [220]:
x , y = sym.symbols('x y', real=True)
z = x + sym.I*y
f = (z**6) + ((((2*np.sqrt(2)+1)*k*(q**2))/(4*(l**2)*w))**2)*(z**2) - ((((2*np.sqrt(2)+1)*k*(q**2))/(4*(l**2)*w))**2)
#f = (z**6) + ((((1+2*np.sqrt(2))*27)/382)**2)*(z**2) - ((((1+2*np.sqrt(2))*27)/382)**2)
F = sym.Matrix([sym.re(f),sym.im(f)])
J = F.jacobian(sym.Matrix([x,y]))
Fn = sym.lambdify([x,y],F,'numpy')
Jn = sym.lambdify([x,y],J,'numpy')

N = 6
x = np.linspace(-1, 1, N)
y = np.linspace(-1.5, 1.5, N)

R = []
for i in range(N):
    for j in range(N):
        r = np.array([x[i],y[j]])
        R.append(r)

Roots = GetRoots(R,Fn,Jn)
Roots

In [221]:
for i in Roots:
    plt.scatter(i[0],i[1])

plt.axhline(0, color='black')
plt.axvline(0, color='black')
plt.xlabel('Re')
plt.ylabel('Im')

Text(0, 0.5, 'Im')

Como sabemos que:

$\displaystyle u = \sin{\left(\theta \right)}$

$\displaystyle 0 \geq \theta \geq \pi/2 , \theta \in \mathbb{R}$

Entonces solo consideraremos la raiz real positiva.

In [222]:
teta = (np.arcsin(Roots[-1][0])*180)/np.pi
round(teta,2)

$% \usepackage{gensymb}$
a) $\displaystyle \theta \approx 37\degree$

b) $\displaystyle \left[ \left( -0.6003712, \  0.0\right), \  \left( -0.3677575, \  -0.5616657\right), \  \left( -0.3677575, \  0.5616657\right), \  \left( 0.3677575, \  -0.5616657\right), \  \left( 0.3677575, \  0.5616657\right), \  \left( 0.6003712, \  0.0\right)\right]$

Donde el primer elemento de cada tupla es la parte real y la segunda es la parte imaginaria.

c) $\displaystyle 0.6003712$

# 23.

a)

In [223]:
A = np.array([[0.2,0.1,1.,1.,0.],
              [0.1,4.,-1.,1.,-1.],
              [1.,-1.,60.,0.,-2.],
              [1.,1.,0.,8.,4.],
              [0.,-1.,-2.,4.,700.]])

b = np.array([1.,2.,3.,4.,5.])

b)

In [224]:
def Metodo_gradiente_conjugado (A,b,x0,tolerancia=0.01):

    r0 = np.matmul(A,x0) - b
    p0 = -1.*r0
    k = 0
    error = 1

    while error > tolerancia and k < 10:
        
        r0_ = r0
        alpha = -1.*(np.dot(r0.T,p0))/np.dot(p0.T,np.matmul(A,p0))
        #print(alpha)
        x0 = x0 + np.dot(alpha,p0)
        print(x0)
        r0 = np.matmul(A,x0) - b
        #print(r0)
        beta = (np.dot(r0.T,np.matmul(A,p0)))/np.dot(p0.T,np.matmul(A,p0))
        #print(beta)
        p0 = -1.*r0 + np.dot(beta,p0)
        #print(p0)
        k += 1
        error = np.max( np.abs(r0-r0_) )

    return x0,k

c)

In [225]:
x,k = Metodo_gradiente_conjugado(A,b,np.array([1.,0.5,-1.,-0.5,0.]))
x

[ 1.03512891  0.49063229 -0.02419691 -0.39851648  0.08587067]
[ 1.03946078  0.49057465  0.05204942 -0.38296965  0.00942789]
[1.17181318 0.53581274 0.0398494  0.28860847 0.00637383]
[1.33629034 0.32965228 0.03343898 0.30019881 0.00599257]
[ 7.85971308  0.42292641 -0.07359224 -0.54064302  0.01062616]
[ 7.85971308  0.42292641 -0.07359224 -0.54064302  0.01062616]


array([ 7.85971308,  0.42292641, -0.07359224, -0.54064302,  0.01062616])

In [226]:
k

Llega a la solución en 5 iteraciones porque la última no cuenta.

In [227]:
np.linalg.solve(A,b)

array([ 7.85971308,  0.42292641, -0.07359224, -0.54064302,  0.01062616])